<a href="https://colab.research.google.com/github/yuyu990116/transformers_tutorials/blob/main/P2_sentence_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#句子相似度检测，使用回归（均方误差）loss

In [1]:
from google.colab import drive
drive.mount("/content/drive")
import os
os.chdir("/content/drive/MyDrive/nlp")
!pip install datasets
!pip install -U accelerate
!pip install -U transformers
!pip install -U evaluate
import evaluate
from datasets import Dataset,load_dataset
from transformers import AutoTokenizer,AutoModelForSequenceClassification,DataCollatorWithPadding,Trainer,TrainingArguments,pipeline


Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 29.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00


In [3]:
datasets = load_dataset("clue",name="afqmc")
model = AutoModelForSequenceClassification.from_pretrained("hfl/chinese-macbert-base",num_labels=1)
#num_labels=1就会自动成为回归模型而不是分类模型，这是transformers中BertForSequenceClassification源码实现的
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/3861 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/34334 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4316 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [4]:
datasets

DatasetDict({
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3861
    })
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 34334
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 4316
    })
})

In [5]:
train_dataset=datasets["train"]
train_dataset

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 34334
})

In [6]:
train_dataset[0]

{'sentence1': '蚂蚁借呗等额还款可以换成先息后本吗',
 'sentence2': '借呗有先息到期还本吗',
 'label': 0,
 'idx': 0}

In [7]:
def data_process(examples):
  tokenized_examples=tokenizer(examples["sentence1"],examples["sentence2"],max_length=128,truncation=True)
  tokenized_examples["labels"]=[float(label) for label in examples["label"]]
  return tokenized_examples
tokenized_datasets=datasets.map(data_process,batched=True,remove_columns=datasets["train"].column_names)
tokenized_datasets

Map:   0%|          | 0/3861 [00:00<?, ? examples/s]

Map:   0%|          | 0/34334 [00:00<?, ? examples/s]

Map:   0%|          | 0/4316 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3861
    })
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 34334
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4316
    })
})

In [8]:
print(tokenized_datasets["train"][0])

{'input_ids': [101, 6010, 6009, 955, 1446, 5023, 7583, 6820, 3621, 1377, 809, 2940, 2768, 1044, 2622, 1400, 3315, 1408, 102, 955, 1446, 3300, 1044, 2622, 1168, 3309, 6820, 3315, 1408, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': 0.0}


In [14]:
acc = evaluate.load("accuracy")
f1 = evaluate.load("f1")
def metric(results):
  predictions,labels=results
  predictions=[int(pred>0.5) for pred in predictions]
  labels = [int(label)for label in labels]
  acc_scores = acc.compute(predictions=predictions,references=labels)
  f1_scores = f1.compute(predictions=predictions,references=labels)
  acc_scores.update(f1_scores)
  return acc_scores

In [10]:
train_args = TrainingArguments(output_dir="./cross_model",      # 输出文件夹
                per_device_train_batch_size=4,  # 训练时的batch_size
                per_device_eval_batch_size=32,  # 验证时的batch_size
                logging_steps=10,                # log 打印的频率
                evaluation_strategy="steps",     # 评估策略
                eval_steps=20,
                save_steps=10,           # 保存策略
                learning_rate=2e-5,              # 学习率
                weight_decay=0.01,               # weight_decay
                metric_for_best_model="f1",      # 设定评估指标
                num_train_epochs=1)

In [11]:
# train_args = TrainingArguments(output_dir="./cross_model",      # 输出文件夹
#                                per_device_train_batch_size=32,  # 训练时的batch_size
#                                per_device_eval_batch_size=32,  # 验证时的batch_size
#                                logging_steps=10,                # log 打印的频率
#                                evaluation_strategy="epoch",     # 评估策略
#                                save_strategy="epoch",           # 保存策略
#                                save_total_limit=3,              # 最大保存数
#                                learning_rate=2e-5,              # 学习率
#                                weight_decay=0.01,               # weight_decay
#                                metric_for_best_model="f1",      # 设定评估指标
#                                load_best_model_at_end=True)     # 训练完成后加载最优模型

In [15]:
trainer = Trainer(model=model,
                  args=train_args,
                  train_dataset=tokenized_datasets["train"],
                  eval_dataset=tokenized_datasets["validation"],
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                  compute_metrics=metric)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(tokenized_datasets["test"])

In [30]:
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/nlp/cross_model/checkpoint-30",num_labels=1)
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [31]:
result = pipe({"text": "我喜欢北京", "text_pair": "北京真好"}, function_to_apply="none") #回归模型就需要加上functiontoappply=none
result["label"] = "相似" if result["score"] > 0.5 else "不相似"
result

{'label': '不相似', 'score': 0.3960985541343689}

In [ ]:
# #如果是分类模型：
# model.config.id2label = {0: "不相似", 1: "相似"}
# pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)
# result = pipe({"text": "我喜欢北京", "text_pair": "天气怎样"})